# temporary Script

In [1]:
from multidimvis_main import *

In [11]:
G = nx.read_edgelist('input/ppi_elist.txt',data=False)

d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
d_names_do = {y:x for x,y in d_do_names.items()}

In [15]:
DM_CC = pd.read_csv('output_csv/DistanceMatrix_goCC_Dataframe_Human.csv', index_col=0)

In [ ]:
DM_BP = pd.read_csv('output_csv/DistanceMatrix_goBP_Dataframe_Human.csv', index_col=0)

In [ ]:
DM_MF = pd.read_csv('output_csv/DistanceMatrix_goMF_Dataframe_Human.csv', index_col=0)

In [88]:
DM_Disease = pd.read_csv('output_csv/DistanceMatrix_Disease_Dataframe_Human.csv', index_col=0)

________
### 3D tSNE

In [89]:
prplxty3d = 50 # range: 5-50 / default = 30.#
density3d = 1.2 # default 12.
l_rate3d = 200 # default 200.
steps3d = 250 #min 250

metric = 'euclidean'
#metric = 'correlation'
#metric = 'cosine'

In [93]:
%%time

#feature = 'CC'
#DM = DM_CC

#feature = 'BP'
#DM = DM_BP

#feature = 'MF'
#DM = DM_MF

feature = 'DISEASE'
DM = DM_Disease
l_genes = list(DM_Disease.index)

#tsne_posG3d = embed_tsne_3D(G, DM, prplxty3d, density3d, l_rate3d, steps3d, metric)
#df_3Dportrait_tsne = pd.DataFrame(tsne_posG3d).T

#df_3Dportrait_tsne.to_csv(r'output_csv/3DPortrait_tsne_'+feature+'_Human_'+'lrate_'+l_rate3d+'steps_'+steps3d+'_'+metric+'.csv', index = True)

CPU times: user 592 µs, sys: 17 µs, total: 609 µs
Wall time: 614 µs


In [85]:
%%time 
tsne = embed_tsne_3D_(DM,  prplxty3d, density3d, l_rate3d, steps3d, metric)

CPU times: user 6min 15s, sys: 12.7 s, total: 6min 27s
Wall time: 4min 42s


In [94]:
%%time 
posG = get_posG_3D(l_genes, tsne)

CPU times: user 4.02 ms, sys: 7 µs, total: 4.02 ms
Wall time: 4.04 ms


### VISUALIZATION SETTINGS

In [97]:
l_features = l_genes 

In [153]:
def draw_node_degree_3D(G, l_genes, scalef):
    x = 20
    ring_frac = (x-1.)/x

    deg = dict(G.degree())
    
    l_size = {}
    for i in l_genes:
        for k,v in deg.items():
            if i == k:
                R = scalef * (1+v**1.5)
                r = ring_frac * R
                l_size[i] = r
    
    return l_size

In [154]:
size3d = draw_node_degree_3D(G, l_genes, 20)

edge_color = 'grey'
edge_width = 0.1

opacity_nodes = 1
opacity_edges = 0.3
node_edge_col = 'dimgrey'

+ HUBS

In [130]:
cutoff = 3 # number of considered hub nodes 
d_hub_nodes = identify_hubs(d_deghubs, d_clos, d_betw, cutoff)
colours = color_majornodes(G, d_hub_nodes)
edge_color = color_majornodes_outgoingedges(G, d_hub_nodes)

+ CLUSTERS

In [163]:
df_posG = pd.DataFrame(posG).T

In [165]:
X = df_posG # df with coordinates

In [188]:
number_of_clusters = 5

In [189]:
%%time 
kmeans_c = KMeans(number_of_clusters).fit(X)
centroids = kmeans_c.cluster_centers_

kmeans = KMeans(number_of_clusters).fit_transform(X)

CPU times: user 1.23 s, sys: 735 ms, total: 1.96 s
Wall time: 356 ms


In [191]:
kmeans

array([[21.57535602,  8.28704628, 16.23670497, 15.84579196, 18.31642468],
       [17.74483829,  8.553617  , 18.71883491, 14.36246427, 10.5064708 ],
       [13.74908726,  6.99638696,  7.73511221,  9.01768706, 13.58004682],
       ...,
       [16.48856644, 14.88304799, 11.12910582,  6.08967396, 16.45975796],
       [14.16068878, 14.76240495, 14.68759671,  1.97618975, 10.52811092],
       [ 7.9588385 , 18.80713901, 17.77835941, 21.81875528, 14.2987256 ]])

### 3D Portrait LAYOUTS 

In [131]:
def get_trace_nodes_(posG, info_list, color_list, size):

    key_list=list(posG.keys())
    trace = pgo.Scatter3d(x=[posG[key_list[i]][0] for i in range(len(key_list))],
                           y=[posG[key_list[i]][1] for i in range(len(key_list))],
                           z=[posG[key_list[i]][2] for i in range(len(key_list))],
                           mode = 'markers',
                           text = info_list,
                           hoverinfo = 'text',
                           #textposition='middle center',
                           marker = dict(
                color = color_list,
                size = size,
                symbol = 'circle',
                line = dict(width = 1.0,
                        color = color_list)
            ),
        )
    
    return trace

In [147]:
def get_trace_edges_(G, posG, color_list):
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in G.edges():
        for node in posG.keys():
            if node in edge:
                x0, y0, z0 = posG[edge[0]]
                x1, y1, z1 = posG[edge[1]]
                edge_x.append(x0)
                edge_x.append(x1)
                edge_x.append(None)
                edge_y.append(y0)
                edge_y.append(y1)
                edge_y.append(None)
                edge_z.append(z0)
                edge_z.append(z1)
                edge_z.append(None)

    trace_edges = pgo.Scatter3d(
                        x = edge_x, 
                        y = edge_y, 
                        z = edge_z,
                        mode = 'lines', hoverinfo='none',
                        line = dict(width = 0.5, color = color_list),
                        opacity = 0.3
                )
    
    return trace_edges

In [133]:
%%time
tsne_nodes = get_trace_nodes_(posG, l_features, colours, size3d)

CPU times: user 907 ms, sys: 5.63 ms, total: 913 ms
Wall time: 915 ms


In [150]:
%%time
tsne_edges = get_trace_edges_(G,posG, edge_color)

CPU times: user 2min 11s, sys: 281 ms, total: 2min 11s
Wall time: 2min 12s


In [151]:
tsne_edges

Scatter3d({
    'hoverinfo': 'none',
    'line': {'color': [lightgrey, lightgrey, lightgrey, ..., lightgrey, lightgrey,
                       lightgrey],
             'width': 0.5},
    'mode': 'lines',
    'opacity': 0.3,
    'x': [],
    'y': [],
    'z': []
})

In [155]:
# tsne_data = [tsne_edges, tsne_nodes]
tsne_data = [tsne_nodes]

In [156]:
print('Perplexity:', prplxty3d)
print('Early Exaggeration:', density3d)
print('Learning rate:', l_rate3d)
print('Iterations:', steps3d)

fig = pgo.Figure()
for i in tsne_data:
    fig.add_trace(i)
fig.update_layout(template='plotly_dark', showlegend=False, width=2000, height=2000,
                  scene=dict(
                    aspectratio=dict(
                        x=1,
                        y=1,
                        z=1
                    ),
                    camera=dict(
                        center=dict(
                            x=0,
                            y=0,
                            z=0
                        ),
                        eye=dict(
                            x=1.96903462608,
                            y=-1.09022831971,
                            z=0.405345349304
                        ),
                        up=dict(
                            x=0,
                            y=0,
                            z=1
                        )
                    ),
                    dragmode="turntable",
                    annotations=[dict(
                        showarrow=True,
                        x=0,
                        y=-10,
                        z=10,
                        text=feature,
                        xanchor="right",
                        ay=-100,
                        ax=-100,
                        opacity=0.8,
                        arrowhead=0,
                        arrowwidth=0.5,
                        arrowcolor="white",
                    )]
                ))
py.iplot(fig)

plotly.offline.plot(fig, filename = 'output_plots/3Dportrait_tsne_'+feature+'_'+metric+'_'+organism+'.html', auto_open=False)

Perplexity: 50
Early Exaggeration: 1.2
Learning rate: 200
Iterations: 250


NameError: name 'organism' is not defined